# End-to-end Multi-class Dog Breed Classification

This notebook builds an en-to-end multi-class image classifier using TensorFlow 2.0 and TensorFlow Hub.

## 1. Problem
Identifying the breed of a dog given an image of a dog.

When I'm sitting at the cafe and I take a photo of a dog, I want to know what breed of dog it is.

## 2. Data

The data being used is from Kaggle's dog breed identification competition.

## 3. Evaluation
The evaluation is a file with prediction probabilities for each dog breed of each test image.

## 4. Features
Some information about the data:
* Dealing with images: Unstructured Data, best to use Deep Learning / Transfer Learning
* There are 120 breeds of dogs, hence 120 classes
* There are around 10k + images in the test and training set
* Training set has been provided with labels for each image, and the label for test set is to be predicted.

In [ ]:
from google.colab import userdata
from google.colab import drive
import os

# Load the drive
drive.mount('/content/drive')

# Change Directory
%cd "/content/drive/MyDrive/Colab Notebooks/data"

In [ ]:
# !pip install tensorflow tensorflow-hub keras tf-keras -q

In [ ]:
# Unpack the dataset
# !unzip "/content/drive/MyDrive/Colab Notebooks/data/dog-breed-identification.zip" -d "/content/drive/MyDrive/Colab Notebooks/data/"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

print(tf.__version__)
print(hub.__version__)

# Check for GPU availability
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")


## Getting our data ready (turning into Tensors)
With all machine learning models, our data has to be in numerical format. Turning our images into Tensors (numerical representations).

In [ ]:
# Checkout the labels of our data
import pandas as pd
labels_csv = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/labels.csv")

labels_csv.describe()

In [ ]:
labels_csv.head()

In [ ]:
# Count images per class
import matplotlib.pyplot as plt

labels_csv["breed"].value_counts().plot.bar(figsize=(20, 10))
plt.show()

In [ ]:
round(labels_csv["breed"].value_counts().median(), 2)

In [ ]:
from IPython.display import Image

Image('train/fff43b07992508bc822f33d8ffd902ae.jpg')

## Getting images and their labels
Let's get a list of all of our image file pathnames

In [ ]:
labels_csv.head()

In [ ]:
# Create pathnames from Image ID's
import os

ROOT_PATH = r"/content/drive/MyDrive/Colab Notebooks/data/train"
labels_csv["pathname"] = ROOT_PATH + "/" + labels_csv["id"]
labels_csv.head()


In [ ]:
pathname_list = list(labels_csv["pathname"] + ".jpg")
labels_csv.drop("pathname", axis=1, inplace=True)

pathname_list[:5]

In [ ]:
labels_csv.head()

In [ ]:
# Check number of filename matches image files
import os

print(len(os.listdir(ROOT_PATH)) == len(pathname_list))

In [ ]:
import numpy as np

labels = np.array(labels_csv["breed"])
labels[:5]

In [ ]:
len(labels) == len(pathname_list)

In [ ]:
# Find unique label values
unique_breeds = np.unique(labels)

In [ ]:
len(unique_breeds)

In [ ]:
# Turn a single label into an array of booleans
print(labels[0])
labels[0] == unique_breeds

In [ ]:
array_of_boolean_labels = [x == unique_breeds for x in labels]
array_of_boolean_labels[:2]

In [ ]:
len(array_of_boolean_labels)

In [ ]:
# Turning boolean array into integers
print(labels[0])
print(np.where(unique_breeds == labels[0])) # index where label occurs
print(array_of_boolean_labels[0].argmax())
print(array_of_boolean_labels[0].astype(int))

In [ ]:
array_of_integers = np.array([x.argmax() for x in array_of_boolean_labels])

In [ ]:
array_of_integers[0]

In [ ]:
one_hot = np.eye(len(unique_breeds))[array_of_integers]
one_hot[0]

In [ ]:
# Creating our train / validation / test split
from sklearn.model_selection import train_test_split

X = pathname_list
y = array_of_boolean_labels

NUM_IMAGES = 200

X_train, X_valid, y_train, y_valid = train_test_split(
    X[:NUM_IMAGES],
    y[:NUM_IMAGES],
    test_size=0.2,
    random_state=42
    )


# Preprocessing Images (turning Images into Tensors)
To preprocess images into Tensors -> need a function that does:
1. Take image filepath as input
2. Use TensorFlow to read the file and save it to a variable
3. Turn our image (a jpg) into Tensors
4. Normalize the image (convert color channel values from 0-255 to 0-1)
5. Resize the image to be a shape of (224, 224)
6. Return the modified image

In [ ]:
# Convert image to NumPy array
from matplotlib.pyplot import imread

image = imread(pathname_list[0])
image

In [ ]:
# Convert NumPy array to tensor
tf.constant(image)[:2]

In [ ]:
# Define image size
IMG_SIZE = 224

# Create a function for preprocessing image
def process_image(image_path):
  """
  Takes an image file path and turns it into a Tensor.
  """

  # Read in an image file
  image = tf.io.read_file(image_path)

  # Turn jpg image to numerical Tensor with 3 colour channels (RGB)
  image = tf.image.decode_jpeg(image, channels=3)

  # Convert the colour channel values from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)

  # Resize the image to desired value (224,224)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])

  return image

# Mini-Batch

Why turn our data into batches?
> When processing large sets of images / data in one go, entire dataset may not fit into the memory, hence pass them in batches.

In [ ]:
# Create a function to return a tuple (image, label)

def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

In [ ]:
# Define the batch size, 32 is default
BATCH_SIZE = 32

# Create a function to turn data into batches
def create_data_batches(X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (X) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle if it's validation data.
  Also accepts test data as inpu (no labels).
  """
  # If the data is test dataset, we don't have labels
  if test_data:
    print("Creating test data batches")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X))) # only filepaths (no labels)
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  # If the data is a valid dataset, we don't need to shuffle it
  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),  # filepaths
                                               tf.constant(y))) # labels
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch
  else:
    print("Creating training batches...")
    # Turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    # Shuffling pathnames and labels before mapping image processor function is faster than shuffling images
    data = data.shuffle(buffer_size=len(X))

    # Create (image,label) tuples (this also turns the image path in a preprocessed image)
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)

  return data_batch

In [ ]:
# Create training and validation data batches
train_data = create_data_batches(X_train, y_train)
valid_data = create_data_batches(X_valid, y_valid, valid_data=True)

In [ ]:
# Check different attributes of our data batches
train_data.element_spec, valid_data.element_spec

## Visualizing Data Batches

Data are now in batches, however, these can be a little hard to comprehend, visualising it...

In [ ]:
import matplotlib.pyplot as plt

# Create a function for viewing images in a data batch
def show_25_images(images, labels):
  """
  Displays a plot of 25 images and their labels from a data batch.
  """
  # Setup the figure
  plt.figure(figsize=(10, 10))
  # Loop through 25 (for displaying 25 images)
  for i in range(25):
    # Create subplots (5 rows, 5 columns)
    ax = plt.subplot(5,5,i+1)
    # Display an image
    plt.imshow(images[i])
    # Add the image label as the title
    plt.title(unique_breeds[labels[i].argmax()])
    # Turn the grid lanes off
    plt.axis("off")

In [ ]:
# Generator to iterator
train_images, train_labels = next(train_data.as_numpy_iterator())
# Visualize the data in a batch
show_25_images(train_images, train_labels)

In [ ]:
# Validation set
val_images, val_labels = next(valid_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

## Building a model

Before a model is built, few things to note:
* Input shape
* Output shape
* URL of the model (Transfer Learning)


In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # batch, height, width, colour channels

# Setup output shape of our model
OUTPUT_SHAPE = len(unique_breeds)

# Setup model URL from TensorFlow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"

## Tensorflow Keras API

Now we've got our inputs, outputs and model ready to go.
Let's put them together into a Keras deep learning model.

Knowing this, let's create a function which:
* Takes the input shape, output shape and the model we've chosen as parameters
* Defines the layers in a Keras model in sequential fashion
* Compiles the model
* Builds the model
* Returns the model

In [ ]:
def create_model(img_size=224, output_shape=OUTPUT_SHAPE):
    # Build MobileNetV2 without the top layer
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(img_size, img_size, 3),
        include_top=False,
        weights='imagenet'
    )

    # Build your classifier on top
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(output_shape, activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

In [ ]:
# Create the model
model = create_model()
model.summary()

## Creating Callbacks
Callbacks are helper functions a model can use during training to do such things as save its progress, check its progress or stop training early if a model stops improving.

Two callbacks,
1. TensorBoard which helps track model progress
2. For early stopping which prevents our model from training for too long

### TensorBoard callback

There are three required steps:
1. Load the TensorBoard notebook extension
2. Create a TensorBoard callback which is able to save logs to a directory and pass it to our model's `fit()` function.
3. Visualize our models training logs with the `%tensorboard` magic function (we'll do this after model training).

In [ ]:
# Load TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import datetime

# Create TensorBoard callback
def create_tensorboard_callback():
  """
    Create a log directory for storing TensorBoard logs
  """
  logdir = os.path.join("/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)

In [ ]:
# Early Stopping Callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3
)

### Training a model (on subset of data)
Our first model is only going to train on 1000 images, to validate the pipeline.

In [ ]:
NUM_EPOCHS = 10 #@param {type:"slider", min:10, max:1000}

In [ ]:
# Check GPU availability
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")

# Function to test a model
* Create a model using `create_model()`
* Setup a TensorBoard callback using `create_tensorboard_callback()`
* Call the fit `fit()` function on our model passing it the training data, validation data, number of epochs to train for (`NUM_EPOCHS`) and the callbacks we'd like to use
* Return the model

In [ ]:
# Build a function to train and return a trained model
def train_model():
  """
  Trains a given model and returns the trained version.
  """

  # Create a model
  model = create_model()

  # Create new TensorBoard session everytime we train a model
  tensorboard = create_tensorboard_callback()

  # Fit the model to the data passing it the callbacks we created
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=valid_data,
            validation_freq=1,
            callbacks=[tensorboard, early_stopping])

  # Return the fitted model
  return model

In [ ]:
# Fit the model to the data
model = train_model()

In [ ]:
# Checking the TensorBoard logs
%tensorboard --logdir /logs

### Make predictions on the validation data

In [ ]:
predictions = model.predict(valid_data, verbose=1)
predictions

In [ ]:
predictions.shape

In [ ]:
np.sum(predictions[0])

In [ ]:
index=0
print(predictions[0])
print(f"Max value (probability of prediction): {np.max(predictions[0])}")
print(f"Sum: {np.sum(predictions[index])}")
print(f"Max index: {np.argmax(predictions[index])}")
print(f"Predicted label: {unique_breeds[np.argmax(predictions[index])]}")

In [ ]:
# Turn prediction probabilities into their respective label
def get_pred_label(prediction_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """
  return unique_breeds[np.argmax(prediction_probabilities)]

# Get a predicted label based on an array of prediction probabilities
pred_label = get_pred_label(predictions[3])
pred_label

# Validation data is still in a batch dataset, so unbatch them to make predictions on the validation images and compare them to the validation labels (truth labels).

In [ ]:
def unbatch_dataset(data):
  # Unbatch the data
  images_ = []
  labels_ = []

  # Loop through unbatched data
  for image,label in data.unbatch().as_numpy_iterator():
    images_.append(image)
    labels_.append(unique_breeds[np.argmax(label)])

  return images_, labels_

In [ ]:
# Unbatch
val_images, val_labels = unbatch_dataset(valid_data)

In [ ]:
val_images[0], val_labels[0]

## Visualize predictions
* Take an array of prediction probabilities, an array of ground truth, and an array of images and integers
* Convert the prediction probabilities to a predicted label.
* Plot the predicted label, its predicted probability, the truth label and the target image on a single plot.

In [ ]:
# Visualisation
def plot_pred(prediction_probabilities, labels, images, n=2):
  """
  View the prediction, ground truth label and image for sample n.
  """
  pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]

  # Get the pred label
  pred_label = get_pred_label(pred_prob)

  # Plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  # Change the colour of the title depending on right or wrong
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"

  plt.title("{} {:2.0f}% {}".format(pred_label, np.max(pred_prob)*100, true_label), color=color)

In [ ]:
plot_pred(prediction_probabilities=predictions, labels=val_labels, images=val_images, n=3)

# Confidence Threshold

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
  """
  Plus the top 10 highest prediction confidences along with the truth label for sample n.
  """
  pred_prob, true_label = prediction_probabilities[n], labels[n]

  # Getthe predicted label
  pred_label = get_pred_label(pred_prob)

  # Find the top 10 prediction confidence indexes
  top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]

  # Find the top 10 prediction confidence values
  top_10_pred_values = pred_prob[top_10_pred_indexes]

  # Find the top 10 prediction labels
  top_10_pred_labels = unique_breeds[top_10_pred_indexes]

  # Setup plot
  top_plot = plt.bar(np.arange(len(top_10_pred_labels)),
                     top_10_pred_values,
                     color="grey")
  plt.xticks(np.arange(len(top_10_pred_labels)),
             labels=top_10_pred_labels,
             rotation="vertical")

  # Change color of true label
  if np.isin(true_label, top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
  else:
    pass

In [ ]:
plot_pred_conf(prediction_probabilities=predictions, labels=val_labels, n=3)

In [ ]:
# Check out a few predictions and their different values
i_multiplier = 0
num_rows = 3
num_cols = 2
num_images = num_rows * num_cols
plt.figure(figsize=(10 * num_cols, 5 * num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_pred(predictions, val_labels, val_images, i+i_multiplier)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=predictions, labels=val_labels, n=i+i_multiplier)
plt.tight_layout(h_pad=1.0)
plt.show()

# Confusion Matrix

In [ ]:
# Confusion Matrix
# import itertools
# from sklearn.metrics import confusion_matrix

# def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10,8)):



# Save the model

In [ ]:
def save_model(model, suffix=None):
  """
  Saves a given model in a models directory and appends a suffix (string).
  """
  # Create a model directory pathname with current time
  model_dir = os.path.join("models", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  model_path = model_dir + ".h5" # save format of model
  print(f"Saving model to: {model_path}")

  model.save(model_path)
  return model_path

# Load the model

In [ ]:
def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

  return model

In [ ]:
# Save model
save_model(model, suffix="mobilenetv2-Adam")

In [ ]:
# Load the model
loaded_model = load_model('models/20260203-034454.h5')

# Prediction on Test Set

In [ ]:
# Load test image filename
test_path = 'test/'
test_filenames = [test_path + fname for fname in os.listdir(test_path)]
len(test_filenames)

In [ ]:
# Create test data batch
test_data = create_data_batches(test_filenames, test_data=True)

In [ ]:
test_data

In [ ]:
# Make predictions on test data batch using the loaded full model (DRY Run)
small_test_data = test_data.take(5)  # 5 batches only
test_predictions = loaded_model.predict(small_test_data, verbose=1)

In [ ]:
# Save predictions (NumPy array) to csv file
np.savetxt("preds_log.csv", test_predictions, delimiter=",")

In [ ]:
# Load predictions from csv file
test_predictions = np.loadtxt("preds_log.csv", delimiter=",")

In [ ]:
test_predictions[0]